### ライブラリのインポート

In [176]:
import pandas as pd
from IPython.display import display

### Google Driveのマウント

In [177]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### pd.to_datetime()の基本的な使い方

In [178]:
# 年月日の8桁の文字列を読み込む
pd.to_datetime("20201218")

Timestamp('2020-12-18 00:00:00')

In [179]:
# 年月日時分の12桁の文字列を読み込む
pd.to_datetime("202012180150")

Timestamp('2020-12-18 01:50:00')

In [180]:
# 時分が150になっており、11桁しかないので読み込めない
pd.to_datetime("20201218150")

OverflowError: ignored

In [181]:
# 数値データで年月日時分を設定する
time = 202012181050

# 数値データは正しく読み込めない
pd.to_datetime(time)

Timestamp('1970-01-01 00:03:22.012181050')

In [182]:
# 文字列に変換する必要がある
pd.to_datetime(str(time))

Timestamp('2020-12-18 10:50:00')

### Pandas の DataFrame型とpd.to_datetime()

In [183]:
# モックデータをマウントしたGoodle Driveの直下に保存
df = pd.read_csv("/content/drive/MyDrive/MOCK_DATA.csv")

In [184]:
display(df.head())
display(df.dtypes)

,id,date1,time1,date2,time2
0,1,20201103,162,20201214,442
1,2,20201109,2030,20201214,500
2,3,20201129,1330,20201203,1360
3,4,20201119,1014,20201209,310
4,5,20201124,634,20201215,429


id       int64
date1    int64
time1    int64
date2    int64
time2    int64
dtype: object

In [185]:
# 数値データは正しく読みこめない
pd.to_datetime(df["date1"])

0     1970-01-01 00:00:00.020201103
1     1970-01-01 00:00:00.020201109
2     1970-01-01 00:00:00.020201129
3     1970-01-01 00:00:00.020201119
4     1970-01-01 00:00:00.020201124
                   ...             
995   1970-01-01 00:00:00.020201113
996   1970-01-01 00:00:00.020201124
997   1970-01-01 00:00:00.020201128
998   1970-01-01 00:00:00.020201123
999   1970-01-01 00:00:00.020201105
Name: date1, Length: 1000, dtype: datetime64[ns]

In [186]:
# astype(str) で文字列データに変換して読み込む
pd.to_datetime(df["date1"].astype(str))

0     2020-11-03
1     2020-11-09
2     2020-11-29
3     2020-11-19
4     2020-11-24
         ...    
995   2020-11-13
996   2020-11-24
997   2020-11-28
998   2020-11-23
999   2020-11-05
Name: date1, Length: 1000, dtype: datetime64[ns]

In [187]:
# date カラムと time カラムを結合して年月日時分のデータを作成する
df_date_time = df["date1"].astype(str) + df["time1"].astype(str)
df_date_time

0       20201103162
1      202011092030
2      202011291330
3      202011191014
4       20201124634
           ...     
995     20201113130
996     20201124425
997    202011281744
998     20201123643
999    202011051011
Length: 1000, dtype: object

In [188]:
# 1行目は時分が 1274（12時74分）になっているなど、正しくないので読み込めない
# 2行目は自分が 188 1時88分
pd.to_datetime(df_date_time)

OverflowError: ignored

### 前処理

In [189]:
# 文字列に変換
df["date1"] = df["date1"].astype(str)
df["time1"] = df["time1"].astype(str)
df["date2"] = df["date2"].astype(str)
df["time2"] = df["time2"].astype(str)

In [190]:
# 例えば 1時50分が 150 ではなく 0150 となるよう、時間データが2桁～3桁のものについて、先頭に0を付けて4桁に修正する

# まず、スライスで時間部分を抽出する
display(df["time1"].str[:-2])

# まず、スライスで分部分を抽出する
display(df["time1"].str[-3:-1])

0       1
1      20
2      13
3      10
4       6
       ..
995     1
996     4
997    17
998     6
999    10
Name: time1, Length: 1000, dtype: object

0      16
1      03
2      33
3      01
4      63
       ..
995    13
996    42
997    74
998    64
999    01
Name: time1, Length: 1000, dtype: object

In [191]:
# 内包表現で、timeが2桁の場合、3桁の場合、それぞれについて先頭に必要なだけ 0 を追加する

# 2桁の場合
df["time1"] = ["00" + i if len(i)==2 else i for i in df["time1"]]
df["time2"] = ["00" + i if len(i)==2 else i for i in df["time2"]]

# 3桁の場合
df["time1"] = ["0" + i if len(i)==3 else i for i in df["time1"]]
df["time2"] = ["0" + i if len(i)==3 else i for i in df["time2"]]

In [192]:
# 時分が4桁になっているのでスライスの指定が変わる点に注意

# スライスで時間部分を抽出する
display(df["time1"].str[:2])

# スライスで分部分を抽出する
display(df["time1"].str[-2:])

0      01
1      20
2      13
3      10
4      06
       ..
995    01
996    04
997    17
998    06
999    10
Name: time1, Length: 1000, dtype: object

0      62
1      30
2      30
3      14
4      34
       ..
995    30
996    25
997    44
998    43
999    11
Name: time1, Length: 1000, dtype: object

In [193]:
df["date1"] + df["time1"]

0      202011030162
1      202011092030
2      202011291330
3      202011191014
4      202011240634
           ...     
995    202011130130
996    202011240425
997    202011281744
998    202011230643
999    202011051011
Length: 1000, dtype: object

In [194]:
# 分が60以上になっている場合、時間を+1、分を-60 と処理
# ただし時間が23時のときは24時となってしまい、認識できないため、別途修正が必要だが、今回は無視する
df["time1"] = [str(int(i[:2])+1) + str(int(i[-2:])-60) if int(i[-2:])>=70 else i for i in df["time1"]]
df["time1"] = [str(int(i[:2])+1) + "0" + str(int(i[-2:])-60) if int(i[-2:])>=60 else i for i in df["time1"]]

df["time2"] = [str(int(i[:2])+1) + str(int(i[-2:])-60) if int(i[-2:])>=70 else i for i in df["time2"]]
df["time2"] = [str(int(i[:2])+1) + "0" + str(int(i[-2:])-60) if int(i[-2:])>=60 else i for i in df["time2"]]

In [195]:
# もう一度、内包表現で、timeが2桁の場合、3桁の場合、それぞれについて先頭に必要なだけ 0 を追加する

# 2桁の場合
df["time1"] = ["00" + i if len(i)==2 else i for i in df["time1"]]
df["time2"] = ["00" + i if len(i)==2 else i for i in df["time2"]]

# 3桁の場合
df["time1"] = ["0" + i if len(i)==3 else i for i in df["time1"]]
df["time2"] = ["0" + i if len(i)==3 else i for i in df["time2"]]

In [196]:
# 前処理後、改めてto_datetime()で読み込む
df_date_time1 = df["date1"] + df["time1"]
df_date_time2 = df["date2"] + df["time2"]

df["date_time1"] = pd.to_datetime(df_date_time1)
df["date_time2"] = pd.to_datetime(df_date_time2)

In [197]:
# 完成
df.head()

,id,date1,time1,date2,time2,date_time1,date_time2
0,1,20201103,0202,20201214,0442,2020-11-03 02:02:00,2020-12-14 04:42:00
1,2,20201109,2030,20201214,0500,2020-11-09 20:30:00,2020-12-14 05:00:00
2,3,20201129,1330,20201203,1400,2020-11-29 13:30:00,2020-12-03 14:00:00
3,4,20201119,1014,20201209,0310,2020-11-19 10:14:00,2020-12-09 03:10:00
4,5,20201124,0634,20201215,0429,2020-11-24 06:34:00,2020-12-15 04:29:00


### date_time1 と date_time2 の差を取る

In [200]:
df["elapsed_time"] = df["date_time2"] - df["date_time1"]

# timedelta型になる
df["elapsed_time"]

0     41 days 02:40:00
1     34 days 08:30:00
2      4 days 00:30:00
3     19 days 16:56:00
4     20 days 21:55:00
            ...       
995   43 days 09:56:00
996   34 days 03:56:00
997    6 days 06:46:00
998   37 days 19:26:00
999   54 days 10:06:00
Name: elapsed_time, Length: 1000, dtype: timedelta64[ns]

In [202]:
# 経過時間を分に修正する
df["elapsed_minute"] = df["elapsed_time"].apply(lambda x: x.seconds//60)

In [203]:
# 経過時間を日に修正する
df["elapsed_day"] = df["elapsed_time"].apply(lambda x: x.days)

In [204]:
df.head()

,id,date1,time1,date2,time2,date_time1,date_time2,elapsed_time,elapsed_minute,elapsed_day
0,1,20201103,0202,20201214,0442,2020-11-03 02:02:00,2020-12-14 04:42:00,41 days 02:40:00,160,41
1,2,20201109,2030,20201214,0500,2020-11-09 20:30:00,2020-12-14 05:00:00,34 days 08:30:00,510,34
2,3,20201129,1330,20201203,1400,2020-11-29 13:30:00,2020-12-03 14:00:00,4 days 00:30:00,30,4
3,4,20201119,1014,20201209,0310,2020-11-19 10:14:00,2020-12-09 03:10:00,19 days 16:56:00,1016,19
4,5,20201124,0634,20201215,0429,2020-11-24 06:34:00,2020-12-15 04:29:00,20 days 21:55:00,1315,20


In [205]:
df["elapsed_total_minute"] = df["elapsed_day"] * 24 * 60 + df["elapsed_minute"]

In [206]:
df.head()

,id,date1,time1,date2,time2,date_time1,date_time2,elapsed_time,elapsed_minute,elapsed_day,elapsed_total_minute
0,1,20201103,0202,20201214,0442,2020-11-03 02:02:00,2020-12-14 04:42:00,41 days 02:40:00,160,41,59200
1,2,20201109,2030,20201214,0500,2020-11-09 20:30:00,2020-12-14 05:00:00,34 days 08:30:00,510,34,49470
2,3,20201129,1330,20201203,1400,2020-11-29 13:30:00,2020-12-03 14:00:00,4 days 00:30:00,30,4,5790
3,4,20201119,1014,20201209,0310,2020-11-19 10:14:00,2020-12-09 03:10:00,19 days 16:56:00,1016,19,28376
4,5,20201124,0634,20201215,0429,2020-11-24 06:34:00,2020-12-15 04:29:00,20 days 21:55:00,1315,20,30115
